In [26]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import time
from geopy.distance import great_circle
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn import metrics
%matplotlib inline

# This package will output the execution time of each cell. Pretty neat!
warnings.filterwarnings('ignore')
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
warnings.filterwarnings('default')
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 281 ms


### Let's bring our nice, clean data with rountrips computer

In [27]:
training = pd.read_csv('data/training_simple.csv')
testing = pd.read_csv('data/testing_simple.csv')

# training_full = pd.read_csv('data/training_clean.csv')
# testing_full = pd.read_csv('data/testing_clean.csv')

time: 5.55 s


### Pull out the target vectors of the training and testing data

In [28]:
training_answers = np.array(training['roundtrip'])
testing_answers = np.array(testing['roundtrip'])

time: 50.9 ms


### Remove a few columns we don't want in our feature vector
### Our feature vector is [ start station id, gender, age, day of week, minute (minutes since 12:00am) ]

In [29]:
del training['end station id']
del training['tripduration']
del training['roundtrip']
del testing['end station id']
del testing['tripduration']
del testing['roundtrip']
training.head()

,start station id,gender,age,day of week,minute
0,128,1,31,0,0
1,438,1,18,0,0
2,383,1,22,0,1
3,361,1,34,0,1
4,284,1,39,0,3


time: 1.22 s


### Looks good! Now let's pull out our samples to train and test

In [30]:
training_samples = training.as_matrix()
testing_samples = testing.as_matrix()
training_samples

array([[ 128,    1,   31,    0,    0],
       [ 438,    1,   18,    0,    0],
       [ 383,    1,   22,    0,    1],
       ..., 
       [3119,    1,   27,    0, 1438],
       [ 518,    1,   45,    0, 1438],
       [ 336,    1,   33,    0, 1439]])

time: 20.7 ms


### Now let's pull in the scikit-learn Gaussian Naive Bayes module

In [38]:
clf = GaussianNB()
clf.fit(training_samples, training_answers)
predictions = clf.predict(testing_samples)

time: 1.56 s


### Time to test! We want a  streamlined way to cross-validate our model. Scikit learn to the rescue!

In [39]:
scores = cross_val_score(clf, testing_samples, testing_answers, cv=5)

time: 8.43 s


In [40]:
print("Accuracy: %0.4f%%" % (scores.mean() * 100))

Accuracy: 98.1944%
time: 1.71 ms


### Pretty darn accurate! Let's try a different method (Naive Bayes is so 2012)

In [37]:
predicted = cross_val_predict(clf, testing_samples, testing_answers, cv=10)
metrics.accuracy_score(testing_answers, predicted) 

0.98194448155173797

time: 16.8 s
